In [8]:
# Ячейка 1: Импорт библиотек и настройка окружения
from typing import List
import os
import json
import requests
import re
from dotenv import load_dotenv
import pandas as pd
import sqlite3
import psycopg2
from psycopg2.extras import RealDictCursor
from IPython.display import display, Markdown

# Загрузка переменных окружения
dotenv_path = os.path.join("..", "src\.env")
load_dotenv(dotenv_path)


print("Библиотеки загружены и окружение настроено")

Библиотеки загружены и окружение настроено


In [9]:
def execute_sql_query(sql_query):
    """
    Executes an SQL query in Postgres database and returns the results in a dictionary.

    Parameters
    ----------
    sql_query : str
        SQL query string.

    Returns
    -------
    dict
        A dictionary with two keys
        - 'result' as a pandas DataFrame containing the query results,
        - 'error' with an error message if the execution failed.
    """
    host = os.getenv("PG_STUDENT_HOST")
    port = os.getenv("PG_STUDENT_PORT")
    dbname = os.getenv("PG_STUDENT_DBNAME")
    user = os.getenv("PG_STUDENT_USER")
    password = os.getenv("PG_STUDENT_PASSWORD")

    # Проверка переменных окружения
    env_vars = {
        "PG_STUDENT_HOST": host,
        "PG_STUDENT_PORT": port,
        "PG_STUDENT_DBNAME": dbname,
        "PG_STUDENT_USER": user,
        "PG_STUDENT_PASSWORD": password,
    }
    missing = [name for name, value in env_vars.items() if not value]
    if missing:
        return {
            "result": None,
            "error": f"Missing environment variables: {', '.join(missing)}",
        }

    conn = None
    cursor = None
    try:
        port = int(port)
        conn = psycopg2.connect(
            host=host, port=port, dbname=dbname, user=user, password=password
        )
        cursor = conn.cursor(cursor_factory=RealDictCursor)
        cursor.execute(sql_query)

        # Получаем данные и названия колонок
        columns = [desc[0] for desc in cursor.description] if cursor.description else []
        rows = cursor.fetchall() if cursor.description else []

        # Создаём DataFrame
        df = pd.DataFrame(rows, columns=columns) if columns else pd.DataFrame()

        return {"result": df, "error": None}

    except Exception as e:
        return {"result": None, "error": str(e)}

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [10]:
query = """
    SELECT 
        table_name, 
        column_name, 
        data_type
    FROM information_schema.columns
    WHERE 
        table_schema = 'public'
        AND table_name IN (
            'categories',
            'countries',
            'merchants',
            'order_items',
            'orders',
            'products',
            'shipping_carriers',
            'users'
        )
    ORDER BY table_name, ordinal_position;
"""

result = execute_sql_query(query)

In [11]:
print(result)

{'result':            table_name          column_name                    data_type
0          categories                   id                      integer
1          categories                 name            character varying
2          categories   parent_category_id                      integer
3           countries                   id                      integer
4           countries                 name            character varying
5           merchants                   id                      integer
6           merchants         country_code                      integer
7           merchants               status            character varying
8           merchants        merchant_name            character varying
9           merchants              address                         text
10          merchants          website_url            character varying
11          merchants         phone_number            character varying
12          merchants                email           

In [12]:
def build_dbml_schema() -> str:
    """
    Generates a DBML schema for specified tables
    in a PostgreSQL database schema using a static SQL query.
    This version formats the output as per the provided DBML schema example,
    replacing 'double precision' data type with 'double'
    for dbdiagram.io compatibility.

    Parameters
    ----------
    table_names : list
        List of table names for which the DBML schema should be generated.

    Returns
    -------
    str
        DBML schema as a string for the specified tables in the schema.
    """
    tables_json = os.getenv("TABLES", "[]")
    table_names = json.loads(tables_json)
    if not table_names:
        return ""

    # Формируем безопасный список имён таблиц (поскольку он фиксированный и контролируемый)
    # Экранируем одинарные кавычки на всякий случай, хотя в нашем случае их нет
    safe_table_names = []
    for name in table_names:
        if not name.replace("_", "").replace("-", "").isalnum():
            raise ValueError(f"Invalid table name: {name}")
        safe_table_names.append(name)

    table_names_str = "', '".join(safe_table_names)
    query = f"""
        SELECT 
            table_name, 
            column_name, 
            data_type
        FROM information_schema.columns
        WHERE 
            table_schema = 'public'
            AND table_name IN ('{table_names_str}')
        ORDER BY table_name, ordinal_position;
    """

    result = execute_sql_query(query)

    if result["error"] is not None:
        raise RuntimeError(f"Failed to fetch schema: {result['error']}")

    df = result["result"]

    if df.empty:
        return ""

    # Сопоставление типов PostgreSQL → DBML (особенно double precision → double)
    def map_type(pg_type):
        pg_type = pg_type.lower()
        if pg_type == "double precision":
            return "double"
        elif pg_type == "integer":
            return "int"
        elif pg_type == "character varying":
            return "varchar"
        elif pg_type == "timestamp without time zone":
            return "timestamp"
        elif pg_type == "timestamp with time zone":
            return "timestamptz"
        elif pg_type == "boolean":
            return "bool"
        elif pg_type == "bigint":
            return "bigint"
        elif pg_type == "smallint":
            return "smallint"
        elif pg_type == "numeric" or pg_type == "decimal":
            return "decimal"
        else:
            return pg_type

    # Группируем по таблицам
    dbml_lines = []
    current_table = None

    for _, row in df.iterrows():
        table = row["table_name"]
        column = row["column_name"]
        dtype = map_type(row["data_type"])

        if table != current_table:
            if current_table is not None:
                dbml_lines.append("}")
            dbml_lines.append(f"Table {table} {{")
            current_table = table

        dbml_lines.append(f"  {column} {dtype}")

    if current_table is not None:
        dbml_lines.append("}")

    return "\n".join(dbml_lines)

In [13]:
# Ячейка 2: Определение схемы БД (можно менять для тестирования)

DB_SCHEMA = build_dbml_schema()
print(DB_SCHEMA)

Table categories {
  id int
  name varchar
  parent_category_id int
}
Table countries {
  id int
  name varchar
}
Table merchants {
  id int
  country_code int
  status varchar
  merchant_name varchar
  address text
  website_url varchar
  phone_number varchar
  email varchar
  logo_url varchar
  created_at timestamp
}
Table order_items {
  id int
  order_id int
  product_id int
  quantity int
  price double
  sum double
}
Table orders {
  id int
  user_id int
  status varchar
  created_at timestamp
  total_sum double
  shipping_address text
  billing_address text
  payment_method varchar
  payment_status varchar
  shipping_carrier_id int
}
Table products {
  id int
  name varchar
  description text
  brand varchar
  color varchar
  weight double
  dimensions varchar
  rating int
  merchant_id int
  price int
  created_at timestamp
  category_id int
}
Table shipping_carriers {
  id int
  name varchar
  tracking_url varchar
}
Table users {
  id int
  full_name varchar
  email varchar
  

In [14]:
# Ячейка 3: Функция для запросов к YandexGPT
def yandex_gpt_query(user_query):
    """
    Sends a user query to YandexGPT via Yandex Cloud API using an API key.
    """
    API_KEY = os.getenv("YANDEX_API_KEY")
    FOLDER_ID = os.getenv("YANDEX_FOLDER_ID")

    if not API_KEY:
        return {
            "status": "failure",
            "answer": "",
            "error": "Missing environment variable: YANDEX_API_KEY",
        }
    if not FOLDER_ID:
        return {
            "status": "failure",
            "answer": "",
            "error": "Missing environment variable: YANDEX_FOLDER_ID",
        }

    model_uri = f"gpt://{FOLDER_ID}/yandexgpt/latest"
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {API_KEY}",
        "x-folder-id": FOLDER_ID,
    }

    data = {
        "modelUri": model_uri,
        "completionOptions": {
            "temperature": 0.3,
            "maxTokens": 2000,
        },
        "messages": [
            {
                "role": "system",
                "text": "You are an assistant",
            },
            {
                "role": "user",
                "text": user_query,
            },
        ],
    }

    try:
        response = requests.post(
            url, headers=headers, data=json.dumps(data), timeout=30
        )

        if response.status_code == 200:
            try:
                result = response.json()
                answer_text = result["result"]["alternatives"][0]["message"]["text"]
                return {"status": "success", "answer": answer_text, "error": ""}
            except (KeyError, IndexError, TypeError) as e:
                return {
                    "status": "failure",
                    "answer": "",
                    "error": f"Unexpected response format: {str(e)}",
                }
        else:
            try:
                error_detail = (
                    response.json().get("error", {}).get("message", "Unknown error")
                )
            except Exception:
                error_detail = response.text or f"HTTP {response.status_code}"
            return {
                "status": "failure",
                "answer": "",
                "error": f"API request failed with status {response.status_code}: {error_detail}",
            }

    except requests.exceptions.RequestException as e:
        return {"status": "failure", "answer": "", "error": f"Request failed: {str(e)}"}
    except Exception as e:
        return {
            "status": "failure",
            "answer": "",
            "error": f"Unexpected error: {str(e)}",
        }


print("Функция yandex_gpt_query определена")

Функция yandex_gpt_query определена


In [15]:
# Ячейка 4: Функция генерации промпта
def generate_prompt(user_query, schema_data):
    """
    Generates a prompt for LLM, including the database schema and the user query.
    """

    system_prompt = """You are a SQL expert that converts natural language questions to PostgreSQL queries.
You MUST use ONLY the tables and columns from the provided database schema.
If the user's request cannot be answered using the available schema, explain why clearly.

DATABASE SCHEMA (DBML format):
{schema_data}

IMPORTANT NOTES ABOUT THE SCHEMA:
- Table "products" has columns: id, name, description, brand, color, weight, dimensions, rating, merchant_id, price, created_at, category_id
- Table "orders" has columns: id, user_id, status, created_at, total_sum, shipping_address, billing_address, payment_method, payment_status, shipping_carrier_id
- Table "users" has columns: id, full_name, email, username, phone_number, last_login_at, avatar_url, created_at, country_code
- Table "categories" has columns: id, name, parent_category_id
- Table "countries" has columns: id, name
- Table "merchants" has columns: id, country_code, status, merchant_name, address, website_url, phone_number, email, logo_url, created_at
- Table "order_items" has columns: id, order_id, product_id, quantity, price, sum
- Table "shipping_carriers" has columns: id, name, tracking_url

RULES:
1. Use only the tables and columns mentioned above
2. For PostgreSQL syntax use CURRENT_DATE for date operations
3. Use proper JOIN syntax with table aliases
4. If the request is impossible with this schema, explain why

EXAMPLES:

1. User: "Show expensive Nike products"
   SQL: "SELECT * FROM products WHERE brand = 'Nike' AND price > 500;"

2. User: "Find USA customers' orders"
   SQL: "SELECT o.* FROM orders o JOIN users u ON o.user_id = u.id JOIN countries c ON u.country_code = c.id WHERE c.name = 'USA';"

3. User: "Count products per category"
   SQL: "SELECT c.name, COUNT(p.id) FROM categories c LEFT JOIN products p ON c.id = p.category_id GROUP BY c.id, c.name;"

4. User: "How old is Dumbledore?"
   Response: {{"sql": "", "error_description": "Cannot answer - the database contains e-commerce data but no information about Harry Potter characters."}}

Now process this request:""".format(
        schema_data=schema_data
    )

    prompt = """User's request: {user_question}

Return your answer as JSON with exactly these fields:
- "sql": the SQL query (empty string if impossible)
- "error_description": explanation why SQL cannot be generated (empty string if SQL is generated)

Return ONLY the JSON, no other text.

Example of valid response when SQL is possible:
{{"sql": "SELECT * FROM products WHERE price > 100;", "error_description": ""}}

Example of valid response when SQL is NOT possible:
{{"sql": "", "error_description": "The request cannot be converted to SQL because..."}}""".format(
        user_question=user_query
    )

    return system_prompt, prompt


print("Функция generate_prompt определена")

Функция generate_prompt определена


In [16]:
# Ячейка 5: Основная функция преобразования natural language to SQL
def natural_language_to_sql(user_query, schema_data):
    """
    Converts a user query from natural language into an SQL query using LLM.
    """

    # Generate prompts using the existing function
    system_prompt, prompt = generate_prompt(user_query, schema_data)

    # Combine system prompt and user prompt for the LLM
    combined_prompt = f"{system_prompt}\n\n{prompt}"

    # Call the LLM
    llm_response = yandex_gpt_query(combined_prompt)

    # Process the response
    if llm_response["status"] == "failure":
        return {
            "sql": "",
            "error_description": f"LLM API error: {llm_response['error']}",
        }

    answer_text = llm_response["answer"].strip()

    # Try to extract JSON from the response
    try:
        # Clean the response - remove markdown code blocks if present
        cleaned_text = re.sub(r"```json\s*|\s*```", "", answer_text)

        # Look for JSON pattern in the response
        json_match = re.search(
            r"\{[^{}]*\{[^{}]*\}[^{}]*\}|\{[^{}]*\}", cleaned_text, re.DOTALL
        )
        if json_match:
            json_str = json_match.group()
            result = json.loads(json_str)

            # Validate the response structure
            if "sql" in result and "error_description" in result:
                # Clean up the SQL query if present
                sql_query = result["sql"].strip() if result["sql"] else ""
                if sql_query and sql_query.startswith('"') and sql_query.endswith('"'):
                    sql_query = sql_query[1:-1]  # Remove surrounding quotes

                return {
                    "sql": sql_query,
                    "error_description": (
                        result["error_description"].strip()
                        if result["error_description"]
                        else ""
                    ),
                }

        # If we get here, JSON parsing failed or structure is wrong
        return {
            "sql": "",
            "error_description": f"Invalid response format from LLM. Expected JSON with 'sql' and 'error_description' fields. Got: {answer_text}",
        }

    except json.JSONDecodeError as e:
        return {
            "sql": "",
            "error_description": f"Failed to parse JSON response: {str(e)}. Full response: {answer_text}",
        }
    except Exception as e:
        return {
            "sql": "",
            "error_description": f"Unexpected error processing LLM response: {str(e)}",
        }


print("Функция natural_language_to_sql определена")

Функция natural_language_to_sql определена


In [17]:
# Ячейка 6: Тестовые вопросы для проверки
TEST_QUERIES = [
    # Валидные запросы
    "Покажи все продукты дороже 1000 рублей",
    "Найди топ-5 самых дорогих товаров",
    "Покажи все заказы за последние 30 дней",
    "Сколько пользователей из США?",
    "Покажи товары бренда Nike с рейтингом выше 4",
    # Запросы с JOIN
    "Покажи все заказы с информацией о пользователях и странах",
    "Найди товары и их категории",
    "Покажи мерчантов и их страну",
    # Агрегатные запросы
    "Сумма продаж по месяцам",
    "Количество товаров в каждой категории",
    "Средний чек заказов",
    # Невозможные запросы
    "Сколько лет Дамблдору из Гарри Поттера?",
    "Какая погода в Москве?",
    "Когда родился Наполеон?",
]

print("Тестовые вопросы подготовлены")

Тестовые вопросы подготовлены


In [18]:
# Ячейка 7: Функция для красивого вывода результатов
def display_query_result(user_query, result):
    """
    Красиво отображает результат обработки запроса
    """
    display(Markdown(f"### 📝 Запрос: `{user_query}`"))

    if result["sql"]:
        display(Markdown("**✅ SQL запрос сгенерирован:**"))
        display(Markdown(f"```sql\n{result['sql']}\n```"))
    else:
        display(Markdown("**❌ SQL не сгенерирован:**"))
        display(Markdown(f"*{result['error_description']}*"))

    display(Markdown("---"))


def test_single_query(user_query, schema_data=DB_SCHEMA):
    """
    Тестирует один запрос и возвращает результат
    """
    print(f"🔍 Обрабатываю запрос: {user_query}")
    result = natural_language_to_sql(user_query, schema_data)
    display_query_result(user_query, result)
    return result


def run_test_queries(queries=TEST_QUERIES, schema_data=DB_SCHEMA, limit=None):
    """
    Запускает тестовые запросы и показывает результаты
    """
    test_queries = queries[:limit] if limit else queries

    results = []
    for i, query in enumerate(test_queries, 1):
        display(Markdown(f"## Тест {i}/{len(test_queries)}"))
        result = test_single_query(query, schema_data)
        results.append((query, result))

    return results


print("Функции для тестирования подготовлены")

Функции для тестирования подготовлены


In [19]:
# Ячейка 8: Быстрый тест одного запроса
# Меняйте этот запрос для тестирования
quick_test_query = "Покажи топ-5 самых дорогих товаров"
quick_result = test_single_query(quick_test_query)

🔍 Обрабатываю запрос: Покажи топ-5 самых дорогих товаров


### 📝 Запрос: `Покажи топ-5 самых дорогих товаров`

**✅ SQL запрос сгенерирован:**

```sql
SELECT * FROM products ORDER BY price DESC LIMIT 5;
```

---

In [20]:
# Ячейка 9: Запуск всех тестов (можно ограничить количество limit=5)
print("🚀 Запуск полного тестирования...")
all_results = run_test_queries(limit=3)

🚀 Запуск полного тестирования...


## Тест 1/3

🔍 Обрабатываю запрос: Покажи все продукты дороже 1000 рублей


### 📝 Запрос: `Покажи все продукты дороже 1000 рублей`

**✅ SQL запрос сгенерирован:**

...

🔍 Обрабатываю запрос: Покажи все заказы за последние 30 дней


### 📝 Запрос: `Покажи все заказы за последние 30 дней`

**✅ SQL запрос сгенерирован:**

```sql
SELECT * FROM orders WHERE created_at >= CURRENT_DATE - interval '30 days';
```

---

In [21]:
# Ячейка 10: Анализ результатов
def analyze_results(results):
    """
    Анализирует результаты тестирования
    """
    total = len(results)
    successful = sum(1 for _, result in results if result["sql"])
    failed = total - successful

    display(Markdown("## 📊 Статистика тестирования"))
    display(Markdown(f"**Всего запросов:** {total}"))
    display(Markdown(f"**Успешных:** {successful}"))
    display(Markdown(f"**Неудачных:** {failed}"))
    display(Markdown(f"**Успешность:** {successful/total*100:.1f}%"))

    # Показать проблемные запросы
    if failed > 0:
        display(Markdown("### ❌ Проблемные запросы:"))
        for query, result in results:
            if not result["sql"]:
                display(Markdown(f"- `{query}`"))
                display(Markdown(f"  *Ошибка: {result['error_description']}*"))


# Запуск анализа
analyze_results(all_results)

## 📊 Статистика тестирования

**Всего запросов:** 3

**Успешных:** 3

**Неудачных:** 0

**Успешность:** 100.0%

In [ ]:
# Ячейка 11: Интерактивное тестирование
def interactive_test():
    """
    Функция для интерактивного тестирования запросов
    """
    display(Markdown("## 🎯 Интерактивное тестирование"))
    display(Markdown("Вводите запросы на естественном языке для преобразования в SQL"))

    while True:
        user_input = input("\nВведите запрос (или 'quit' для выхода): ").strip()

        if user_input.lower() in ["quit", "exit", "выход"]:
            break

        if user_input:
            result = test_single_query(user_input)

            # Предложить сохранить результат
            save = input("Сохранить этот результат? (y/n): ").strip().lower()
            if save == "y":
                # Здесь можно добавить логику сохранения
                print("✅ Результат сохранен (функциональность можно расширить)")


# Раскомментируйте для использования:
interactive_test()

In [22]:
# Ячейка 12: Отладка и логирование
import datetime


def debug_query(user_query, schema_data=DB_SCHEMA):
    """
    Расширенная функция отладки с подробным логированием
    """
    print(f"🔧 [DEBUG] Начало обработки: {datetime.datetime.now()}")
    print(f"📝 Исходный запрос: {user_query}")

    # Генерация промпта
    system_prompt, prompt = generate_prompt(user_query, schema_data)
    print(f"📋 Длина system_prompt: {len(system_prompt)} символов")
    print(f"📋 Длина prompt: {len(prompt)} символов")

    # Вызов LLM
    combined_prompt = f"{system_prompt}\n\n{prompt}"
    print("🤖 Вызов YandexGPT...")
    llm_response = yandex_gpt_query(combined_prompt)

    print(f"📡 Статус ответа: {llm_response['status']}")
    if llm_response["status"] == "success":
        print(f"📄 Длина ответа: {len(llm_response['answer'])} символов")
        print(f"📄 Ответ LLM: {llm_response['answer'][:500]}...")
    else:
        print(f"❌ Ошибка: {llm_response['error']}")

    # Обработка результата
    result = natural_language_to_sql(user_query, schema_data)
    print(f"🎯 Конечный результат:")
    print(f"   SQL: {result['sql']}")
    print(f"   Ошибка: {result['error_description']}")

    return result


# Пример использования отладки
debug_result = debug_query("Покажи все продукты дороже 500 рублей")

🔧 [DEBUG] Начало обработки: 2025-10-11 16:39:52.275675
📝 Исходный запрос: Покажи все продукты дороже 500 рублей
📋 Длина system_prompt: 3163 символов
📋 Длина prompt: 562 символов
🤖 Вызов YandexGPT...
📡 Статус ответа: success
📄 Длина ответа: 85 символов
📄 Ответ LLM: ```
{"sql": "SELECT * FROM products WHERE price > 500;", "error_description": ""}
```...
🎯 Конечный результат:
   SQL: SELECT * FROM products WHERE price > 500;
   Ошибка: 


In [23]:
# Ячейка 13: Экспорт результатов
def export_results(results, filename="dwh_assistant_results.json"):
    """
    Экспортирует результаты тестирования в JSON файл
    """
    export_data = []
    for query, result in results:
        export_data.append(
            {
                "query": query,
                "sql": result["sql"],
                "error_description": result["error_description"],
                "timestamp": datetime.datetime.now().isoformat(),
            }
        )

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Результаты экспортированы в {filename}")


# Экспорт результатов (раскомментируйте при необходимости)
export_results(all_results)

✅ Результаты экспортированы в dwh_assistant_results.json
